## 1. Play the game

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import random
import numpy as np
import pandas as pd

In [3]:
from kaggle_environments import make, evaluate

# ゲーム環境の構築
# set=Trueはデバッグモード
env = make("connectx", debug=True)

# 利用可能な既定のエージェント
print(list(env.agents))

Loading environment football failed: No module named 'gfootball'
['random', 'negamax']


- エージェントはrandom, negamaxの2種類
- randomエージェントは、可能な戦略のうち、ランダムに選択する（connect fourの場合は、列にdiscが入る余裕があるときに、その列を選択可能; 最大でも7列）

In [4]:
# 2人のランダムエージェントがゲームをプレイ
env.run(['random', 'random'])

# show
env.render(mode='ipython')

- ゲームの様子を動画で確認できるし、再生することもできる

- エージェントの定義は、2つの引数obs, configをとり、整数値（7列のゲームなので0~6の値）を返す関数

In [5]:
import random
import numpy as np

- obs:  
    - config.board: 各マスの状態を表すリスト
    - obs.mark: エージェントに割り当てられた番号（1または2の値）
- config:
    - config.columns: 盤面の列数
    - config.rows: 盤面の行数
    - config.inarow: 勝利に必要なピース数

In [6]:
# 戦略をランダムに選択
def agent_random(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# 戦略のうち、真ん中の列を選択
def agent_middle(obs, config):
    return config.columns//2

# 戦略のうち、最も左の列を選択
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return valid_moves[0]

In [7]:
# ゲームをプレイ
env.run([agent_leftmost, agent_random])

# Show
env.render(mode="ipython")

- 1ゲームだけ行うのでは、エージェントがどれだけよいか評価を行うには不十分
- より好ましいのは、複数回ゲームをプレイして、平均勝率を算出

In [8]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # connect four
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # 試合数の半分はエージェント1が先攻          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # 試合数の半分はエーゲンと2が先攻
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    # 試合の様子を出力　
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    # 不当な試合を行った回数（余裕のない列を選択したら不当な試合とされ、余裕のない列を選択したプレイヤーにNone値）
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [9]:
get_win_percentages(agent1=agent_middle, agent2=agent_random)

Agent 1 Win Percentage: 0.72
Agent 2 Win Percentage: 0.01
Number of Invalid Plays by Agent 1: 27
Number of Invalid Plays by Agent 2: 0


In [10]:
get_win_percentages(agent1=agent_leftmost, agent2=agent_random)

Agent 1 Win Percentage: 0.84
Agent 2 Win Percentage: 0.16
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0


In [11]:
# ゲームをプレイ
env.run([agent_middle, agent_random])

# Show
env.render(mode="ipython")

## 2. One step lookahead

- 考えられる盤面の数は約4兆となるため、実用的には盤面の部分集合のみを考え、盤面にスコアを割り当てる
- ヒューリスティックに、隣接する4つのdiscが構成されているグループをみる
    - 1000000 points: 4つのdiscが並んでいる
    - 1 point: 3つのスポットを埋め、残りのスポットが空
    - -100 points: 対戦相手が3つのスポットを埋め、残りのスポットが空
- 一般的に、ヒューリスティック（スコアの与え方）がわからない場合、まずは適当に設計してエージェントと対戦してみるのがよい

In [12]:
# 選択した列にドロップしたときの盤面スコアを計算
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    score = get_heuristic(next_grid, mark, config)
    return score

# score_moveのヘルパー関数: 選択した列にドロップしたときの、次の盤面を計算
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# score_moveのヘルパー関数: 盤面のスコアを計算
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
    return score

# get_heuristicのヘルパー関数: window（縦横斜めの4マス）がヒューリスティック条件を満たすか確認
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# get_heuristicのヘルパー関数: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

In [13]:
# The agent is always implemented as a Python function that accepts two arguments: obs and config
def agent(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next turn
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    r = random.choice(max_cols)
    return r

In [14]:
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx")

# Two random agents play one game round
env.run([agent, "random"])

# Show
env.render(mode="ipython")

In [15]:
get_win_percentages(agent1=agent, agent2="random", n_rounds=100)

Agent 1 Win Percentage: 1.0
Agent 2 Win Percentage: 0.0
Number of Invalid Plays by Agent 1: 0
Number of Invalid Plays by Agent 2: 0
